In [ ]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

In [ ]:
PATH = "result/VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))

In [ ]:
# --- Unstructured pruning experiment ---
import copy
import torch.nn.utils.prune as prune

# helper: apply global unstructured pruning to all conv weights to target sparsity
def apply_unstructured_pruning(model, sparsity=0.8):
    conv_modules = []
    for m in model.modules():
        # support QuantConv2d or nn.Conv2d (QuantConv2d should expose weight)
        if m.__class__.__name__.endswith('Conv2d') or m.__class__.__name__ == 'QuantConv2d':
            if hasattr(m, 'weight') and m.weight is not None:
                conv_modules.append((m, 'weight'))
    # apply global unstructured pruning
    prune.global_unstructured(conv_modules, pruning_method=prune.L1Unstructured, amount=sparsity)
    return model

# reload original trained checkpoint (4-bit quantized) to keep a clean starting point
orig_checkpoint = torch.load(PATH, map_location=device)
orig_state = orig_checkpoint['state_dict']
base_model = VGG16_quant()
base_model.load_state_dict(orig_state, strict=False)
base_model.to(device)
base_model.eval()

# Save a copy before pruning
pruned_model = copy.deepcopy(base_model)
pruned_model.train()

# Apply 80% unstructured pruning globally
target_sparsity = 0.8
pruned_model = apply_unstructured_pruning(pruned_model, sparsity=target_sparsity)

# compute and print overall sparsity
def compute_sparsity(model):
    total = 0
    zero = 0
    for m in model.modules():
        if hasattr(m, 'weight') and m.weight is not None:
            w = m.weight.data.cpu()
            total += w.numel()
            zero += (w == 0).sum().item()
    return zero / total if total>0 else 0

print('Unstructured pruning applied. Sparsity:', compute_sparsity(pruned_model))

# Evaluate after pruning (no finetune)
def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            out = model(data)
            pred = out.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)
    acc = 100. * correct / total
    print('Accuracy: {}/{} ({:.2f}%)'.format(correct, total, acc))
    return acc

baseline_acc = evaluate(base_model, testloader, device)
pruned_acc = evaluate(pruned_model, testloader, device)

# mark for next step: finetuning
print('Baseline (pre-prune) acc:', baseline_acc)
print('Post-unstructured-prune acc (before finetune):', pruned_acc)

In [ ]:
# --- Finetune pruned model (unstructured) ---
# We'll finetune for a few epochs with a reduced LR to recover accuracy
finetune_model = pruned_model
finetune_model.train()

# create optimizer only for non-zero (masked) params: keep mask params intact
optimizer = optim.SGD(filter(lambda p: p.requires_grad, finetune_model.parameters()), lr=0.01, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss().to(device)

num_epochs = 5  # tweak as needed; short for quick recovery
for epoch in range(num_epochs):
    print('Finetune epoch', epoch+1, '/', num_epochs)
    train(trainloader, finetune_model, criterion, optimizer, epoch)
    val_acc = validate(testloader, finetune_model, criterion)
    print('Validation acc after epoch:', val_acc)

# final evaluation
final_acc_unstructured = evaluate(finetune_model, testloader, device)
print('Final unstructured-pruned model accuracy after finetune:', final_acc_unstructured)

# save checkpoint
os.makedirs('result/pruned_unstructured', exist_ok=True)
torch.save({'state_dict': finetune_model.state_dict(), 'accuracy': final_acc_unstructured}, 'result/pruned_unstructured/model_best.pth.tar')
print('Saved unstructured-pruned model to result/pruned_unstructured/model_best.pth.tar')

In [ ]:
# --- Structured pruning experiment (channel pruning) ---
# Reload a fresh copy of the original checkpoint to start structured pruning from the same baseline
struct_model = VGG16_quant()
struct_model.load_state_dict(orig_state, strict=False)
struct_model.to(device)
struct_model.train()

# We'll prune entire output channels of Conv layers using L1 unstructured on channel dimension
# torch.nn.utils.prune.ln_structured with n=1 and dim=0 prunes output channels
def apply_structured_channel_pruning(model, sparsity=0.8):
    for m in model.modules():
        if m.__class__.__name__.endswith('Conv2d') or m.__class__.__name__ == 'QuantConv2d':
            if hasattr(m, 'weight') and m.weight is not None:
                # prune output channels (dim=0) by L1 norm
                prune.ln_structured(m, name='weight', amount=sparsity, n=1, dim=0)
    return model

struct_model = apply_structured_channel_pruning(struct_model, sparsity=target_sparsity)
print('Structured pruning applied. Channel sparsity (approx):', compute_sparsity(struct_model))

# Evaluate before finetune
pre_finetune_struct_acc = evaluate(struct_model, testloader, device)
print('Post-structured-prune acc (before finetune):', pre_finetune_struct_acc)

# Finetune structured-pruned model
struct_finetune_optim = optim.SGD(filter(lambda p: p.requires_grad, struct_model.parameters()), lr=0.01, momentum=0.9, weight_decay=1e-4)
struct_criterion = nn.CrossEntropyLoss().to(device)
num_epochs_struct = 5
for epoch in range(num_epochs_struct):
    print('Structured finetune epoch', epoch+1, '/', num_epochs_struct)
    train(trainloader, struct_model, struct_criterion, struct_finetune_optim, epoch)
    va = validate(testloader, struct_model, struct_criterion)
    print('Validation acc after epoch:', va)

final_acc_structured = evaluate(struct_model, testloader, device)
print('Final structured-pruned model accuracy after finetune:', final_acc_structured)

os.makedirs('result/pruned_structured', exist_ok=True)
torch.save({'state_dict': struct_model.state_dict(), 'accuracy': final_acc_structured}, 'result/pruned_structured/model_best.pth.tar')
print('Saved structured-pruned model to result/pruned_structured/model_best.pth.tar')

# Summary printout
print('SUMMARY:')
print('Baseline acc:', baseline_acc)
print('After unstructured prune (before finetune):', pruned_acc)
print('After unstructured prune + finetune:', final_acc_unstructured)
print('After structured prune (before finetune):', pre_finetune_struct_acc)
print('After structured prune + finetune:', final_acc_structured)


In [ ]:
#### Prune all the QuantConv2D layers' 90% weights with 1) unstructured, and 2) structured manner.



In [ ]:
print(list(model.features[40].named_parameters())) # check whether there is mask, weight_org, ...
print(model.features[40].weight) # check whether there are many zeros

In [ ]:
### Check sparsity ###
mask1 = model.features[40].weight_mask
sparsity_mask1 = (mask1 == 0).sum() / mask1.nelement()

print("Sparsity level: ", sparsity_mask1)

In [ ]:
## check accuracy after pruning

In [ ]:
## Start finetuning (training here), and see how much you can recover your accuracy ##
## You can change hyper parameters such as epochs or lr ##



In [ ]:
## check your accuracy again after finetuning

In [ ]:
## Send an image and use prehook to grab the inputs of all the QuantConv2d layers

class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = dataiter.next()
images = images.to(device)
out = model(images)

In [ ]:
##### Find "weight_int" for features[3] ####
w_bit = 4
weight_q = model.features[3].weight_q
w_alpha = model.features[3].weight_quant.wgt_alpha
w_delta = w_alpha /(2**(w_bit-1)-1)

weight_int = weight_q / w_delta
print(weight_int)

In [ ]:
#### check your sparsity for weight_int is near 90% #####
#### Your sparsity could be >90% after quantization #####
sparsity_weight_int = (weight_int == 0).sum() / weight_int.nelement()
print("Sparsity level: ", sparsity_weight_int)